In [24]:
import findspark
findspark.init()
import pandas as pd
from pyspark import SparkConf, SparkContext
import os
import re
import time
import datetime
import json

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()

conf = SparkConf()
sc = SparkContext.getOrCreate(conf=conf)

In [25]:
K = 50
       
with open('./data_{}_cluster/user_feature_{}.json'.format(K, K), 'r') as inputfile:  
    user_feature_dict = json.load(inputfile)

user_ids = user_feature_dict.keys()
print(len(user_feature_dict))

7087


In [28]:
post_feature = sc.textFile('./post_feature')

post_feature = post_feature.map(lambda line: line.replace('(', ''))\
                .map(lambda line: line.replace(')', ''))\
                .map(lambda line: line.split(', '))\
                .map(lambda line: (line[0].replace("'", ""), line[1:]))

# group post id with the same timestamp
post_publish_time_dict = post_feature.map(lambda info: (info[1][-1], info[0]))\
                    .groupByKey().mapValues(list)\
                    .collectAsMap()

#print(post_publish_time_dict.get('1553817600.0'))
#print(len(post_publish_time_dict))

['2309789', '2310904', '1000917', '1389984', '2311688', '1001410', '2306187', '1324238', '1001776', '1211916', '2311709', '1001582', '2311431', '1317541', '2311630', '1001614', '663148', '1084478', '2311718', '2311211', '1001445', '6602', '2311312', '2311718', '2310713', '1001281', '53903', '5483', '2311476', '1346158', '2310957', '663206', '2311486', '2311482', '70038', '2311474', '2223149', '72226', '2312813', '757178', '2311566', '663338', '2312832', '2272753', '2311436', '663464', '53903', '2313121', '2309146', '2311436', '663442', '663458', '2311828', '2311500', '2283426', '4543', '2311514', '2291315', '663490', '2311515', '2310564', '663450', '765332', '430869', '89143', '2311564', '2311838', '663518', '2311571', '424992', '2311316', '657842', '1273186', '1088397', '91593', '2298923', '2298261', '96851', '2311592', '2311478', '663556', '2310581', '2227860', '663423', '1111397', '1044183', '663587', '2305614', '740078', '85905', '2263429', '2208156', '663401', '2309795', '2289075'

In [30]:
with open('./data_{}_cluster/post_feature_{}.json'.format(K,K), 'r') as inputfile:  
    post_feature_dict = json.load(inputfile)

post_ids = post_feature_dict.keys()
#print(len(post_feature_dict))
#print(post_feature_dict.get('2309789'))



172229
[0.146206657, 0.007333982, -0.193162151, -0.058332917, -0.298346654, -0.529479765, 0.252821563, 0.27732051, -0.015769866, 0.169143362, 0.054258816, 0.055953693, 0.25107081, 0.019315529, 0.018892732, -0.0696173, -0.251590204, -0.004245037, 0.058539802, -0.000402402, 0.098419676, 0.136568456, -0.17824587, 0.134277787, -0.067115486, -0.128589335, 0.090935289, -0.098718996, 0.143428179, 0.291399827, -0.180217687, -0.047158011, 13.0, 1553817600.0]


In [31]:
user_clicklist_train = sc.textFile('./older_data/user_clicklist_train_final')\
                .map(lambda line: tuple(re.findall(r"'(.*?)'", line)))\
                .map(lambda info: (info[0] + '|' + info[1], (info[2:])))

user_clicklist_test = sc.textFile('./older_data/user_clicklist_test_final')\
                .map(lambda line: tuple(re.findall(r"'(.*?)'", line)))\
                .map(lambda info: (info[0] + '|' + info[1], (info[2:])))

#print(user_clicklist_train.take(1))
#print(user_clicklist_test.take(1))

[('122b7665-6fac-4278-97e8-b72f7ccbb5dc|CA', ('1204234', '1279326', '1303559', '1303095', '1294468', '1291345', '1294630', '1300744', '1306010', '1297366', '1306464', '1300534', '1292887', '1304703', '1252819', '1304914', '1302796', '835561', '1306111', '1303479', '1294018', '1306502', '571314', '1230406', '1297326', '1266919', '1319968', '1321695', '822793', '1320028', '1324974', '1316488', '1311850', '1316835', '1231743', '1320001', '1316364', '1252819', '1315364', '1316381', '1310291', '1313806', '1314249', '1311329', '1311003', '1325428', '1316363', '716627', '1253223', '1319976', '1328614', '1248217', '1350109', '1346601', '559815', '1213333', '59859', '1350476', '1327102', '722391', '1327941', '1307546', '1334790', '1306535', '1327545', '1327641', '1326167', '1349870', '1234020', '1341232', '1350470', '1342615', '1348048', '1347829', '739076', '1344368', '1343083', '715619', '1344061', '1341736', '1341574', '1339778', '1047306', '1351678', '1339322', '1183115', '1084445', '135559

In [32]:
# combine train and test clicklist
user_clicklist = user_clicklist_train.union(user_clicklist_test).reduceByKey(lambda x1, x2: x1 + x2)

#print(user_clicklist.take(1))
#print(user_clicklist.count())


[('3597d5b5-01f2-44c4-ae34-d6279970e569|GB', ('1268347', '1297829', '1293070', '1295562', '1287213', '1302627', '1294332', '1296589', '1168912', '1297834', '1296767', '1299417', '1306010', '1302619', '1294350', '1294468', '1283659', '1067073', '1294140', '1204234', '1310068', '1319311', '1311913', '1315364', '1316145', '1323764', '1313436', '1310910', '1306010', '1310606', '1311699', '1310577', '1316364', '1319273', '1322011', '1316363', '1322771', '1323565', '1322292', '1322923', '600580', '1322340', '1287242', '1319394', '1324116', '1322024', '1354479', '1395578', '1377294', '1396814', '1394080', '1382649', '1394216', '1396123', '1393790', '1379588', '1396800', '1415072', '1416598', '424992', '1370260', '1414128', '1416632', '1421021', '1413323', '1231743', '715375', '2189928', '2189903', '1411309', '1316626', '2189710', '1369692', '1423565', '1416724', '2198278', '2198182', '1220679', '410779', '2190074', '2194926', '2196601', '857137', '1258062', '2207887', '2203597', '2199227', '2

In [33]:
# replace user clicks with non-clicks
def replace_with_nonclick(tlist):
    nonclicklist = []
    clicklist = dict((id, 1) for id in tlist)
    for clicked_post in tlist:
        tsp = post_feature_dict.get(clicked_post)[-1]
        sametime_posts = post_publish_time_dict.get(str(tsp))
        if sametime_posts != None:
            for post in sametime_posts:
                if clicklist.get(post) == None and post not in nonclicklist:
                    nonclicklist.append(post)
                    break
    return nonclicklist
        
user_non_clicklist = user_clicklist.map(lambda x: (x[0], replace_with_nonclick(x[1])))

#print(user_non_clicklist.take(2))

[('3597d5b5-01f2-44c4-ae34-d6279970e569|GB', ['2283130', '83753', '1298141', '1304262', '1300162', '1302605', '1291233', '83494', '2203718', '1324718', '2237052', '1300895', '1330104', '1301553', '1293900', '2266230', '637035', '1302307', '1294422', '66230', '2291577', '1341463', '1313297', '88087', '1313451', '1323588', '87303', '86833', '1308633', '1314382', '1312030', '1310653', '1316395', '88919', '1313838', '1316403', '1322207', '1323727', '1322255', '107250', '2311481', '1321941', '1324222', '1319903', '1399122', '1322817', '1403864', '1402340', '1396622', '108057', '1400070', '2257224', '1393652', '1400031', '2290041', '1382200', '1399348', '1420371', '1416486', '2309789', '2310379', '1420370', '1409490', '115551', '2188946', '658439', '2281588', '625317', '2194836', '112520', '1418922', '1249295', '1417198', '1423636', '104994', '1341619', '666286', '1227196', '2308164', '2195826', '626672', '74309', '2303831', '2279074', '2222366', '2217908', '628236', '2200503', '114122', '22

In [34]:
user_non_click = user_non_clicklist.flatMap(lambda x: ((x[0].split('|')[0], p) for p in x[1]))

#print(user_non_click.take(1))
#print(user_non_click.count())

In [35]:
# add user feature
user_feature_nonclick = user_non_click.filter(lambda x: user_feature_dict.get(x[0]) != None)\
                            .map(lambda x: (x[0], x[1], user_feature_dict.get(x[0])))

#print(user_feature_nonclick.take(1))

In [36]:
# add post feature
user_usrfeature_pstfeature = user_feature_nonclick.filter(lambda x: post_feature_dict.get(x[1]) != None)\
                            .map(lambda x: (x[0], x[1], x[2], post_feature_dict.get(x[1])))

#print(user_usrfeature_pstfeature.take(1))

In [37]:
user_usrfeature_pstfeature_train_negative = user_usrfeature_pstfeature\
                .map(lambda x: (x[0], )+(x[1], )+ (-1000, ) + tuple(x[2]) + tuple(x[3][:-2]))


from pyspark.sql.types import Row
from pyspark.sql import SparkSession
from pyspark import SparkContext


spark = SparkSession(sc)

#here you are going to create a function
def f(x):
    d = tuple()
    for i in range(len(x)):
        if i == 0 or i == 1:
            d += (x[i],)
        else:
            d += (float(x[i]),)
    return d

#Now populate that
df = user_usrfeature_pstfeature_train_negative.map(lambda x: f(x)).toDF()


In [38]:
#df.show(1)

In [39]:
df.write.csv('./data_{}_cluster/training_0'.format(K))